Billboard Hot 100 Chart Overview

In [3]:
# Import all libraries
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import time
from datetime import datetime, timedelta
import plotly.express as px

In [4]:
# Generate Saturdays from a specific year until today's date
def generate_saturdays(start_year):
    saturdays = []
    start_date = datetime(start_year, 1, 1)
    end_date = datetime.now()

    # Loop through dates until today
    while start_date <= end_date:
        if start_date.weekday() == 5:
            saturdays.append(start_date.strftime('%Y-%m-%d'))
        start_date += timedelta(days=1)

    return saturdays

# Generate Saturdays starting from 2022 until today
saturdays = generate_saturdays(2024)
print(saturdays)

['2024-01-06', '2024-01-13', '2024-01-20', '2024-01-27', '2024-02-03', '2024-02-10', '2024-02-17', '2024-02-24', '2024-03-02', '2024-03-09', '2024-03-16', '2024-03-23', '2024-03-30', '2024-04-06', '2024-04-13', '2024-04-20', '2024-04-27', '2024-05-04', '2024-05-11', '2024-05-18', '2024-05-25', '2024-06-01', '2024-06-08', '2024-06-15', '2024-06-22', '2024-06-29', '2024-07-06', '2024-07-13', '2024-07-20', '2024-07-27', '2024-08-03', '2024-08-10', '2024-08-17', '2024-08-24', '2024-08-31', '2024-09-07', '2024-09-14', '2024-09-21', '2024-09-28']


In [3]:
driver = webdriver.Chrome()

chart = []

for date_str in saturdays:
    url = f'https://www.billboard.com/charts/hot-100/{date_str}/'
    driver.get(url)
    
    w = WebDriverWait(driver, 15)
    w.until(EC.presence_of_element_located((By.CLASS_NAME,"c-title")))

    try:
    # Extract titles
        title1 = driver.find_elements(By.CLASS_NAME, 'c-title.a-no-trucate.a-font-primary-bold-s.u-letter-spacing-0021.u-font-size-23@tablet.lrv-u-font-size-16.u-line-height-125.u-line-height-normal@mobile-max.a-truncate-ellipsis.u-max-width-245.u-max-width-230@tablet-only.u-letter-spacing-0028@tablet')
        title2 = driver.find_elements(By.CLASS_NAME, 'c-title.a-no-trucate.a-font-primary-bold-s.u-letter-spacing-0021.lrv-u-font-size-18@tablet.lrv-u-font-size-16.u-line-height-125.u-line-height-normal@mobile-max.a-truncate-ellipsis.u-max-width-330.u-max-width-230@tablet-only')
        titles_combined = title1 + title2
        titles = [t.text.strip() for t in titles_combined]

        # Extract ranks
        rank_elements = driver.find_elements(By.CLASS_NAME, 'c-label.a-font-primary-bold-l.u-font-size-32@tablet.u-letter-spacing-0080@tablet')
        ranks = [r.text.strip() for r in rank_elements]

        # Extract singers
        singer1 = driver.find_elements(By.CLASS_NAME, 'c-label.a-no-trucate.a-font-primary-s.lrv-u-font-size-14@mobile-max.u-line-height-normal@mobile-max.u-letter-spacing-0021.lrv-u-display-block.a-truncate-ellipsis-2line.u-max-width-330.u-max-width-230@tablet-only.u-font-size-20@tablet')
        singer2 = driver.find_elements(By.CLASS_NAME, 'c-label.a-no-trucate.a-font-primary-s.lrv-u-font-size-14@mobile-max.u-line-height-normal@mobile-max.u-letter-spacing-0021.lrv-u-display-block.a-truncate-ellipsis-2line.u-max-width-330.u-max-width-230@tablet-only')
        singer_combined = singer1 + singer2
        singers = [s.text.strip() for s in singer_combined]

        # Extract last week's positions and total weeks
        last_week_and_total_weeks = driver.find_elements(By.CLASS_NAME, 'o-chart-results-list__item.a-chart-color.u-width-72.u-width-55@mobile-max.u-width-55@tablet-only.lrv-u-flex.lrv-u-flex-shrink-0.lrv-u-align-items-center.lrv-u-justify-content-center.lrv-u-border-b-1.u-border-b-0@mobile-max.lrv-u-border-color-grey-light.u-background-color-white-064@mobile-max.u-hidden@mobile-max')
        last_week_and_total_weeks_combined = [lp.text.strip() for lp in last_week_and_total_weeks]
        last_week_positions = last_week_and_total_weeks_combined[0::2]
        total_weeks = last_week_and_total_weeks_combined[1::2]

        # Extract peak positions
        peak_position = driver.find_elements(By.CLASS_NAME, 'o-chart-results-list__item.a-chart-bg-color.a-chart-color.u-width-72.u-width-55@mobile-max.u-width-55@tablet-only.lrv-u-flex.lrv-u-flex-shrink-0.lrv-u-align-items-center.lrv-u-justify-content-center.lrv-u-background-color-grey-lightest.lrv-u-border-b-1.u-border-b-0@mobile-max.lrv-u-border-color-grey-light.u-hidden@mobile-max')
        peak_positions = [pp.text.strip() for pp in peak_position]
        peak_positions_adjusted = peak_positions[1::2]

        # Combine all data into the chart list
        for t, r, s, l, p, tw in zip(titles, ranks, singers, last_week_positions, peak_positions_adjusted, total_weeks):
            date = datetime.strptime(date_str, '%Y-%m-%d')  # Convert date string back to datetime
            month = date.strftime("%m")  # Extract the month
            day = date.strftime("%d")  # Extract the day
            year = date.strftime("%Y")  # Extract the year
            chart.append((year, month, day, r, t, s, l, p, tw))

        time.sleep(1)
    
    except:
        pass
    
driver.quit() 

# Print the final chart
print(chart)

[]


In [4]:
# Create DataFrame from chart list
df = pd.DataFrame(chart, columns=['Year', 'Month', 'Day', 'Rank', 'Title', 'Singer(s)', 'Last Week Position', 'Peak Position', 'Weeks on Chart'])

# Convert columns to numeric
df['Rank'] = pd.to_numeric(df['Rank'])
df['Last Week Position'] = pd.to_numeric(df['Last Week Position'], errors='coerce')
df['Peak Position'] = pd.to_numeric(df['Peak Position'], errors='coerce')
df['Weeks on Chart'] = pd.to_numeric(df['Weeks on Chart'], errors='coerce')

# Display the DataFrame
df

,Year,Month,Day,Rank,Title,Singer(s),Last Week Position,Peak Position,Weeks on Chart


Top 10 Songs by Accumulated Weeks at No. 1 (2022-2024)

In [5]:
# Filter for songs that were No. 1
no1_songs = df[df['Rank'] == 1]

# Create a new DataFrame with just the Year, Title, and Singer(s)
# Group by Title and Singer(s) to count the weeks at No. 1
accumulated_weeks_no1 = no1_songs.groupby(['Year', 'Title', 'Singer(s)']).size().reset_index(name='Accumulated Weeks')
# Sort by accumulated weeks at No. 1
accumulated_weeks_no1 = accumulated_weeks_no1.sort_values(by=['Accumulated Weeks'], ascending=False)
# Show the top 10 songs with the most accumulated weeks at No.1
top_10_no1 = accumulated_weeks_no1.head(10)

# Display the final DataFrame
print(top_10_no1)

Empty DataFrame
Columns: [Year, Title, Singer(s), Accumulated Weeks]
Index: []


In [6]:
# Top 10 Songs by Accumulated Weeks at No. 1 (2022-2024)

# Create a bar chart
fig = px.bar(
    top_10_no1,
    x='Title',
    y='Accumulated Weeks',
    hover_data=['Singer(s)'],
    color='Accumulated Weeks',
    title='Top 10 Songs by Most Accumulated Weeks at No. 1 (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    xaxis_title='Songs',
    yaxis_title='Accumulated Weeks at No.1',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        tickfont=dict(size=16),
        tickmode='linear'
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()


Monthly New Songs in the Top 10 (2022-2024)

In [7]:
# Filter for new songs that were in Top 10
top10_songs = df[(df['Rank'] <= 10) & (df['Last Week Position'] > 10)]

# Create a new DataFrame with just the Year, Month, Title, and Singer(s)
# Group by Year and Month to count the number of Top 10 songs
top10_songs_by_month = top10_songs.groupby(['Year', 'Month']).size().reset_index(name='Monthly New Songs in the Top 10')
# Combine Year and Month
top10_songs_by_month['Year-Month'] = top10_songs_by_month['Year'] + '-' + top10_songs_by_month['Month']
# Drop the original Year and Month columns
top10_songs_by_month = top10_songs_by_month.drop(columns=['Year', 'Month'])
# Rearrange the columns so that 'Year and Month' comes first
top10_songs_by_month = top10_songs_by_month[['Year-Month', 'Monthly New Songs in the Top 10']]

# Display the final DataFrame
print(top10_songs_by_month)

Empty DataFrame
Columns: [Year-Month, Monthly New Songs in the Top 10]
Index: []


In [8]:
# Monthly New Songs in the Top 10 (2022-2024)

# Create a line chart
fig = px.line(
    top10_songs_by_month,
    x='Year-Month',
    y='Monthly New Songs in the Top 10',
    color_discrete_sequence=['red'],
    title='Monthly New Songs in the Top 10 (2022-2024)',
)

# Customize the chart
fig.update_traces(hovertemplate='Year-Month: %{x|%Y-%m}<br>Value: %{y}')
fig.update_layout(
    xaxis_title='Year-Month',
    yaxis_title='Counts',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ))

# Show the chart
fig.show()


Top 5 Singers with the most numbers of Top 10 Songs (2022-2024)

In [9]:
# Filter for songs that were in the top 10
top10_songs = df[df['Rank'] <= 10]

# Drop duplicates based on 'Title' and 'Singer(s)' to ensure each song is counted once per artist
unique_top10_songs = top10_songs.drop_duplicates(subset=['Title', 'Singer(s)'])

# Group by Singer(s) to count the number of unique top 10 songs
top10_songs_singers = unique_top10_songs.groupby('Singer(s)').size().reset_index(name='No. of Top 10 Songs')

# Sort by the number of top 10 songs
top10_songs_singers = top10_songs_singers.sort_values(by='No. of Top 10 Songs', ascending=False)

# Show the top 5 singers with the most number of unique top 10 songs
top5_singers = top10_songs_singers.head(5)

# Display the final DataFrame
print(top5_singers)

Empty DataFrame
Columns: [Singer(s), No. of Top 10 Songs]
Index: []


In [10]:
# Top 5 Singers with the most numbers of Top 10 Songs (2022-2024)

# Create a bar chart
fig = px.bar(
    top5_singers,
    x='Singer(s)',
    y='No. of Top 10 Songs',
    color='No. of Top 10 Songs',
    title='Top 5 Singers with The Most Numbers of Top 10 Songs (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    xaxis_title='Singer(s)',
    yaxis_title='No. of Top 10 Songs',
    title_font=dict(size=24),
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()


No.1 Songs with Most Weeks on Chart (2022-2024)

In [11]:
# Filter for songs that reached No. 1
no1_songs = df[df['Rank'] == 1]

# Group the songs by Title and Singer(s) and count the maximum weeks
weeks_count = no1_songs.groupby(['Title', 'Singer(s)'])['Weeks on Chart'].max().reset_index()

# Count how many songs spend each number of weeks on chart
weeks_to_no1_count = weeks_count['Weeks on Chart'].value_counts().reset_index()

# Rename the columns for clarity
weeks_to_no1_count.columns = ['Number of Weeks', 'Count of Songs']

# Sort by the number of weeks in ascending order
weeks_to_no1_count_sorted = weeks_to_no1_count.sort_values(by=['Number of Weeks'], ascending=False)

# Show the top 10 results
top10_weeks_to_no1_count_sorted = weeks_to_no1_count_sorted.head(10)

# Display the final DataFrame
print(top10_weeks_to_no1_count_sorted)


Empty DataFrame
Columns: [Number of Weeks, Count of Songs]
Index: []


In [12]:
# No.1 Songs with Most Weeks on Chart (2022-2024)

# Create a bar chart
fig = px.bar(
    top10_weeks_to_no1_count_sorted,
    x='Number of Weeks',
    y='Count of Songs',
    color_discrete_sequence=['#E48F72'],
    title='No.1 Songs with Most Weeks on Chart (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=24),
    xaxis_title='Number of Weeks',
    yaxis_title='Count of Songs',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16),
        type='category'
    ),
    yaxis=dict(
        tickfont=dict(size=16),
        tickmode='linear'
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()


Songs with the most weeks on chart (2022-2024)

In [13]:
# Group the songs by Title and Singer(s) and count the maximum weeks
weeks_on_chart = df.groupby(['Title', 'Singer(s)'])['Weeks on Chart'].max().reset_index()

# Sort by the number of weeks in ascending order
weeks_on_chart_sorted = weeks_on_chart.sort_values(by=['Weeks on Chart'], ascending=False)

# Show the top 10 results
top10_weeks_on_chart_sorted = weeks_on_chart_sorted.head(10)

# Display the final DataFrame
print(top10_weeks_on_chart_sorted)


Empty DataFrame
Columns: [Title, Singer(s), Weeks on Chart]
Index: []


In [14]:
# Songs with the most weeks on chart (2022-2024)
# Create a bar chart
fig = px.bar(
    top10_weeks_on_chart_sorted,
    x='Title',
    y='Weeks on Chart',
    color='Weeks on Chart',
    hover_data=['Singer(s)'],
    title='Songs with The Most Weeks on Chart (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=24),
    title_x=0.5,
    xaxis_title='Songs',
    yaxis_title='Weeks on Chart',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16),
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()

Distribution of Last Week Positions among Top 10 Songs (2022-2024)

In [15]:
# Filter for songs that reached top 10 and were outside that the week before
top10_songs = df[(df['Rank'] <= 10) & (df['Last Week Position'] > 10)]

# Count how many songs took each number of weeks to get to No. 1
last_week_position = top10_songs['Last Week Position'].value_counts().reset_index()

# Rename the columns for clarity
last_week_position.columns = ['Last Week Position', 'Count of Songs']

# Sort by the number of weeks in ascending order
last_week_position_sorted = last_week_position.sort_values(by=['Last Week Position'], ascending=True)

# Show the top 10 results
last_week_position_sorted = last_week_position_sorted.head(20)

# Display the final DataFrame
print(last_week_position_sorted)

Empty DataFrame
Columns: [Last Week Position, Count of Songs]
Index: []


In [16]:
# Distribution of Last Week Positions among Top 10 Songs (2022-2024)
# Create a histogram
fig = px.histogram(
    last_week_position_sorted,
    x='Last Week Position',
    y='Count of Songs',
    color='Last Week Position',
    title='Distribution of Last Week Positions among Top 10 Songs (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=24),
    title_x=0.5,
    xaxis_title='Last Week Position',
    yaxis_title='Count of Songs',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16),
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()

Peak Position vs. Total Weeks on Chart (2022-2024)

In [17]:
# Group the songs by Title, Singer(s), and Peak Position, and get the maximum weeks
weeks_on_chart = df.groupby(['Title', 'Singer(s)', 'Peak Position'])['Weeks on Chart'].max().reset_index()

# Sort the DataFrame by 'Weeks on Chart' in descending order
weeks_on_chart_sorted = weeks_on_chart.sort_values(by='Weeks on Chart', ascending=False)

# Display the final DataFrame
print(weeks_on_chart_sorted)

Empty DataFrame
Columns: [Title, Singer(s), Peak Position, Weeks on Chart]
Index: []


In [18]:
# Peak Position vs. Total Weeks on Chart (2022-2024)
# Create a scatter plot
fig = px.scatter(
    weeks_on_chart_sorted,
    x='Weeks on Chart',
    y='Peak Position',
    color_discrete_sequence=['skyblue'],
    hover_data=['Title', 'Singer(s)'],
    title='Peak Position vs. Total Weeks on Chart (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=24),
    title_x=0.5,
    xaxis_title='Weeks on Chart',
    yaxis_title='Peak Position',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16),
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()

Longest Running Songs in Top 10 (2022-2024)

In [19]:
# Filter for songs that reached Top 10
top10_songs = df[df['Rank'] <= 10]

# Count how many weeks each song spent in the Top 10, including the singer
weeks_top10_count = top10_songs.groupby(['Title', 'Singer(s)']).size().reset_index(name='Weeks in Top 10')

# Sort by the number of weeks in descending order
weeks_top10_count_sorted = weeks_top10_count.sort_values(by=['Weeks in Top 10'], ascending=False)

# Show the top 5 results
top_weeks_top10 = weeks_top10_count_sorted.head(5)

# Display the final DataFrame
print(top_weeks_top10)

Empty DataFrame
Columns: [Title, Singer(s), Weeks in Top 10]
Index: []


In [20]:
# Longest Running Songs in Top 10 (2022-2024)
# Create a bar chart
fig = px.bar(
    top_weeks_top10,
    x='Title',
    y='Weeks in Top 10',
    color='Weeks in Top 10',
    hover_data=['Singer(s)'],
    title='Longest Running Songs in Top 10 (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=24),
    title_x=0.5,
    xaxis_title='Songs',
    yaxis_title='Weeks in Top 10',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16),
    ),
    yaxis=dict(
        tickfont=dict(size=16)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()

Top 5 No. 1 Songs with the Fewest Weeks to Reach the Top Position (2022-2024)

In [21]:
# Filter for songs that reached No. 1
no1_songs = df[df['Rank'] == 1]

# Find the weeks on chart for No. 1 songs grouped by title and singer
weeks_to_best = no1_songs.groupby(['Title', 'Singer(s)'])['Weeks on Chart'].min()

# Count how many songs took each number of weeks to get to No. 1
weeks_to_no1_count = weeks_to_best.value_counts().reset_index()

# Rename columns for clarity
weeks_to_no1_count.columns = ['Weeks to No. 1', 'Counts']

# Sort by the number of weeks in ascending order
weeks_to_no1_count_sorted = weeks_to_no1_count.sort_values(by='Weeks to No. 1', ascending=True)

# Get the top 5 results
top5_weeks_to_no1_count = weeks_to_no1_count_sorted.head(5)

# Display the final DataFrame
print(top5_weeks_to_no1_count)


Empty DataFrame
Columns: [Weeks to No. 1, Counts]
Index: []


In [22]:
# Top 5 No. 1 Songs with the Fewest Weeks to Reach the Top Position (2022-2024)
# Create a bar chart
fig = px.bar(
    top5_weeks_to_no1_count,
    x='Weeks to No. 1',
    y='Counts',
    color_discrete_sequence=['skyblue'],
    title='Top 5 No. 1 Songs with the Fewest Weeks to Reach the Top Position (2022-2024)',
    width=1000
)

# Customize the chart
fig.update_layout(
    title_font=dict(size=22),
    xaxis_title='Weeks to No. 1',
    yaxis_title='Counts',
    xaxis_title_font=dict(size=20),
    yaxis_title_font=dict(size=20),
    xaxis=dict(
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        tickfont=dict(size=12)
    ),
    hoverlabel=dict(
        font=dict(
            family="Arial",
            size=14
        )
    ),
    bargap=0.2
)

# Show the chart
fig.show()

Monthly New Songs in Hot 100 (2022-2024)

In [23]:
# Filter for new songs
new_songs = df[df['Last Week Position'].isna()]

# Count how many new songs are in each month and year
new_songs_by_month = new_songs.groupby(['Year', 'Month']).size().reset_index(name='Count of New Songs')

# Sort by Year and Month in ascending order
new_songs_by_month_sorted = new_songs_by_month.sort_values(by=['Year', 'Month'], ascending=True)

# Display the final DataFrame
print(new_songs_by_month_sorted)

Empty DataFrame
Columns: [Year, Month, Count of New Songs]
Index: []


In [24]:
# Monthly New Songs in Hot 100 (2022-2024)
# Create the heatmap
fig = px.density_heatmap(new_songs_by_month_sorted,
                          x='Month',
                          y='Year',
                          z='Count of New Songs',
                          title='Monthly New Songs in Hot 100 (2022-2024)',
                          labels={'Count of New Songs': 'Number of New Songs'})

fig.update_layout(
    title=dict(
        text='Monthly New Songs in Hot 100 (2022-2024)',
        font=dict(size=24),  
        x=0.5
    )
)

# Show the heatmap
fig.show()
